# FC25 Rush Detection
## Inference Results Analysis
This notebook aims to analyze the inference results from the FC25 Rush Detection model. The objective is to derive actionable insights for the user controlled players and to make strategic decisions based on these results.
### Inference

In [17]:
import os

model_path = os.path.expanduser("~/Documents/Rush/fc25-rush.mk1.pt")
screenshot_location = os.path.expanduser("../screenshots/jpg/cropped")
image_width = 1920
image_height = 1080
confidence_threshold = 0.1
high_confidence = 0.35

In [ ]:
import os
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

# Load the trained YOLO model
model = YOLO(model_path)

# List all image files in the directory
image_files = [f for f in os.listdir(screenshot_location) if f.endswith(('.jpg', '.jpeg', '.png'))]

# Function to perform inference, display results, and save annotations
def run_inference(image_path):
    # Load image using OpenCV
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB
    
    # Perform inference
    results = model(img_rgb, conf=confidence_threshold)
    
    # TODO: Move the scrubbing up here so that the annotated image only has the scrubbed results

    # Render results on the image
    annotated_img = results[0].plot()  # Annotate the image with results

    # Save annotated image, re-add if needed
    annotated_image_path = os.path.join(os.path.expanduser("~/Documents/Rush/annotated"), os.path.basename(image_path))
    plt.imsave(annotated_image_path, annotated_img)  # Save using imsave

    # Prepare detection results
    detections = []
    for result in results[0].boxes:
        x1, y1, x2, y2 = map(int, result.xyxy[0])  # Extract coordinates
        confidence = result.conf[0].item()
        class_id = int(result.cls[0].item())
        class_name = results[0].names[class_id]

        detection = {
            "class_name": class_name, 
            "class_id": class_id,  # Class ID of detected object
            "points": {
                "x": x1 / img.shape[1],  # Convert to relative coordinates
                "y": y1 / img.shape[0],  # Convert to relative coordinates
                "width": (x2 - x1) / img.shape[1],
                "height": (y2 - y1) / img.shape[0]
            },
            "confidence": confidence
        }
        # Scrub the results. Only keep high confidence detections
        if class_name != "ball" and confidence > high_confidence:
             # Append detection to list
            detections.append(detection)
        # Unless it's the ball or the user controlled player
        elif class_name == "ball" or class_name == "user-controlled-player":
            detections.append(detection)
        else:
            print(detection)
        
    return detections

# Run inference for each image in the directory
for image_file in image_files:
    image_path = os.path.join(screenshot_location, image_file)
    detections = run_inference(image_path)


0: 384x640 1 ball, 4 black-teams, 1 goalie, 1 user-controlled-player, 5 white-teams, 179.6ms
Speed: 1.5ms preprocess, 179.6ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
{'class_name': 'black-team', 'class_id': 1, 'points': {'x': 0.2947916666666667, 'y': 0.37222222222222223, 'width': 0.024479166666666666, 'height': 0.08703703703703704}, 'confidence': 0.12855952978134155}
{'class_name': 'white-team', 'class_id': 4, 'points': {'x': 0.38177083333333334, 'y': 0.12037037037037036, 'width': 0.014583333333333334, 'height': 0.03888888888888889}, 'confidence': 0.12010607868432999}


### Validity
Were the inference results valid